### 0. 라이브러리 호출

In [1]:
import openai
import json
import random
import re
import datetime
import os

### 1. API KEY 입력

In [3]:
openai.api_key = "<YOUR API KEY>" #API KEY

### 2. Data 생성

In [4]:
k=1 #생성해낼 데이터 갯수, ChatGPT API자체 오류 때문에 k는 작은 값으로 여러번 실행하는 것이 효율적
chats=[{} for i in range(k)]
count=0
while chats[-1]=={}:
    try:
        is_free = False
        while not is_free:
            
            ###제목 생성 모델###
            completion = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "무료로 나누거나 파는 것이 아닌, 랜덤으로 정확한 물건의 이름과 그에 맞는 정확한 브랜드 이름을 생성해. 절대 대괄호를 사용해서 토큰을 생성하지마. 반드시 물건 이름과 브랜드 이름을 생성해내야해 "},
                    {"role": "user", "content": "무료 나눔이 아닌 랜덤으로 아무 물건의 중고 거래 게시글 제목을 하나만 생성해줘"}
                    ]
                )
            result = completion.choices[0].message
            title=result["content"]
            #무료가 포함된 제목은 제거
            if "무료" not in title:
                is_free = True
            else:
                print("제목에 무료가 포함되었음. :(")
                
        is_free_2 = False
        while not is_free_2:
            
            ###가격 생성 모델###        
            completion = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "중고거래를 위한 게시글의 제목을 줄거야. 이것에 대해 적당한 판매 희망 가격을 생성해줘. 생성결과는 가격을 정수형으로 출력해줘. (예를 들어, 100000) 그리고 무조건 숫자만 생성해줘, 가격협의 이런건 안돼. 형태는 가격: "},
                    {"role": "user", "content": title}
                    ]
                )
            result = completion.choices[0].message
            price=result["content"]
            price=int(price[4:]) #"가격: "의 형식, 숫자만 추출
            
            ###상품 설명 생성 모델### 
            completion = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "중고거래를 위한 게시글의 제목을 입력하면, 이 제목에 대한 상품 설명을 생성해줘, 해당 물건의 상태(예를들어, 사용감, 손상정도)에 대해서 정확한 수치를 사용해서 언급해. 한국말로 생성해줘."},
                    {"role": "user", "content": title}
                    ]
                )
            accept_content_chat=f"중고거래에서 판매자와 구매자가 주고받는 채팅데이터 전체를 생성해라. 다음과 같은 규칙을 지켜서 대화를 진행하라.  0. 매 채팅마다 [판매자] 또는 [구매자]를 구분해라 1. 구매자가 먼저 대화를 시도한다. 2. 판매물건과 판매희망가, 그리고 물건에 대한 판매자의 설명이 주어진다. 3. 최대한 간단 명료하게, 짧게 대화한다. 한문장에서 두 문장 사이로. 4. 대화는 6번 이상 오고 가야한다. 5. 판매자는 최대한 {price}에 가까운 가격으로 물건을 파는 것을 목표로 하되, 구매자의 조건에 따라 목표 가격의 살짝 아래로 판매할 수 있다. 목표 가격은 언급하지 않는다. 구매자가 괜찮은 조건을 제시하였기 때문에 네고를 수락한다.  6. 네고에 대한 언급은 구매자가 먼저 시작한다.  구매자가 제시하는 거래 희망가격은 판매자가 제시하는 거래 희망가격보다 무조건 작거나 같다. 8. 협상이 아닌, 네고라는 단어를 사용한다. 네고는 협상과 동의어이다. 9. 구매자는 네고가를 제시할 때, 그 근거를 최대한 상품 설명에 나와있는 내용을 이용한다. 10. 거래의 구체적인 방법, 특히 주소, 거래 방식, 거래 날짜, 거래 장소, 계좌, 입금방식, 연락처,개인정보에 대해서는 판매자와 구매자 모두 언급하지 않는다. 상품 정보와 거래가에 대한 내용만 생성한다.  12. 대화의 마무리는 다음과 같다. 판매자 또는 구매자가 마지막으로 최종 네고가를 제시한다. 이에 대해 거래가 성사된 경우 생성된 데이터의 제일 마지막에 반드시 <ACCEPT> 토큰을 대화 맨마지막에 덧붙이고 대화를 종료한다."
            deny_content_chat=f"중고거래에서 판매자와 구매자가 주고받는 채팅데이터 전체를 생성해라.  다음과 같은 규칙을 반드시 지켜서 대화를 진행하라. 0. 서로 거래 가격을 합의하지 못해서 거래가 성사되지 못하고 대화가 종료된다. 1. 매 채팅마다 [판매자] 또는 [구매자]를 구분해라 2. 구매자가 먼저 대화를 시도한다. 2. 판매물건과 판매희망가, 그리고 물건에 대한 판매자의 설명이 주어진다. 3. 최대한 간단 명료하게, 짧게 대화한다. 한문장에서 두 문장 사이로. 4. 대화는 6번 이상 오고 가야한다. 5.구매자가 마지막 까지 판매자가 제안한 가격보다 낮은 가격에서의 거래를 원한다. 6. 네고에 대한 언급은 구매자가 먼저 시작한다. 판매자도 어느정도 네고에 호응하지만, 구매자가 제시한 가격보단 높은 가격에서 거래를 원한다. 따라서 거래가 성사되지 않는다 7. 구매자의 말투가 매우 무례할 수도 있다. 8. 구매자가 무례한 말투를 사용할 경우 판매자는 거래를 단호한 어조로 거절한다. 9. 협상이 아닌, 네고라는 단어를 사용한다. 네고는 협상과 동의어이다. 10. 구매자는 네고가를 제시할 때, 그 근거를 최대한 상품 설명에 나와있는 내용을 이용한다. 11. 거래의 구체적인 방법, 특히 주소, 거래 방식, 거래 날짜, 거래 장소, 계좌, 입금방식, 연락처,개인정보에 대해서는 판매자와 구매자 모두 언급하지 않는다. 상품 정보와 거래가에 대한 내용만 생성한다.  12. 대화의 마무리는 다음과 같다. 판매자 또는 구매자가 마지막으로 최종 네고가를 제시한다. 구매자의 네고가가 판매자의 네고가보다 낮기에 거래가 성사되지 않는다. 생성된 데이터의 제일 마지막에 반드시 <DENY> 토큰을 대화 맨마지막에 덧붙이고 대화를 종료한다."
            result = completion.choices[0].message
            description=result["content"]
            title_description =f"제목: {title} 설명: {description} 희망가격:{price}"

            chats[count]["title"]=title
            chats[count]["description"]=description.replace("\n","")
            chats[count]["price"]=price
            
            ###채팅 데이터 생성 모델###            
            accept_per = random.randint(0,100) #거래 성사, 거절 비율 임의 조절
            if accept_per <70:
                completion = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=[
                        {"role": "system", "content": accept_content_chat},
                        {"role": "user", "content": title_description+"주어진 제목과 판매희망가, 상품설명을 참조해서 채팅데이터 전체를 생성해줘"}
                        ]
                    )
            else:
                completion = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=[
                        {"role": "system", "content": deny_content_chat},
                        {"role": "user", "content": title_description+"주어진 제목과 판매희망가, 상품설명을 참조해서 채팅데이터 전체를 생성해줘"}
                        ]
                    )
            result = completion.choices[0].message
            events=result["content"]
            
        ###생성 데이터 전처리###
            a = re.split(r'(?=\[구매자\]|\[판매자\]|\[ACCEPT\]|\<ACCEPT\>|\[DENY\]|\<DENY\>)', events)
            a = [sentence for sentence in a if sentence != ""]
            a = [sentence.replace('\n', "") for sentence in a]
            chats[count]["result"]=a[-1][1:-1]
            if chats[count]["result"] == "ACCEPT" or chats[count]["result"] == "DENY":
                is_free_2 = True
            else:
                print("결과 토큰 오류")
        parsed_events=[]        
        for sentence in a:
            each_chat={}
            if "구매자" in sentence[:7]:
                each_chat["role"]="구매자"
                each_chat["message"]=sentence[6:]
                parsed_events.append(each_chat)
            elif "판매자" in sentence[:7]:
                each_chat["role"]="판매자"
                each_chat["message"]=sentence[6:] 
                parsed_events.append(each_chat)
            else:
                pass     
        chats[count]["events"]=parsed_events
        count += 1
        print (str(count/k*100),"%")
        
    ###예외처리###   
    except ValueError: #가격 생성 오류
        print("ValueError")
        continue

###데이터 저장###
time=str(datetime.datetime.now())[-6:]
new_file_path = f'/opt/generated_data/{time}_generation_sample.json' #YOUR PATH  
with open(new_file_path, 'w', encoding='utf-8') as f:
    json.dump(chats, f, ensure_ascii=False, indent=4)

100.0 %


### 3. Data 병합

In [ ]:
def merge_json_files(file1, file2, output_file): #file1 + file2 = output_file
    with open(file1, 'r') as f1:
        data1 = json.load(f1)
    with open(file2, 'r') as f2:
        data2 = json.load(f2)
    merged_data = data1 + data2
    with open(output_file, 'w', encoding='utf-8') as output:
        json.dump(merged_data, output, ensure_ascii=False, indent=4)

###디렉토리 내의 모든 파일명 가져오기###
file_list=[]
def get_filenames_in_directory(directory):
    filenames = os.listdir(directory)
    for filename in filenames:
        file_list.append("/opt/generated_data/"+str(filename))
    return file_list

###파일 합치기###
directory_path = '/opt/generated_data'
merge_sus=get_filenames_in_directory(directory_path)
merge_sus
for suspect in merge_sus:
    merge_json_files('/opt/merged.json',suspect,'/opt/merged.json')

### 4. Data 전처리

#### 4.1 Data 확인

In [5]:
file_path = "/opt/final_merge.json"
with open(file_path, "r") as json_file:
    json_data = json_file.read()
    parsed_data = json.loads(json_data)
len(parsed_data)

1000

#### 4.2 Data 가격 정보 추출

In [6]:
last_prices=[]
for k in range(len(parsed_data)):
    price_lists=[]
    
    ###정규표현식을 활용한 가격정보 추출###
    for i in range(len(parsed_data[k]['events'])):
        patterns = [r"\d+,\d+,*\d+",r"\d{3}\d+",r"\d*만\s*\d*천+\s*",r"\d+만(?!\s?\d)"]
        price_list=[parsed_data[k]['events'][i]['role']]
        
        ###1.숫자로 되어있으나 쉼표가 있는 경우###   
        matches=re.findall(patterns[0],parsed_data[k]['events'][i]['message'])
        if matches:
            numbers = matches
            for number in range(len(numbers)):
                numbers[number] = "".join([sublist.replace(",", "") for sublist in numbers[number]])
            price_list.append(numbers)
            
        ###2.단순히 숫자로만 된 가격(4자리 이상)### 
        matches=re.findall(patterns[1],parsed_data[k]['events'][i]['message'])
        if matches:
            numbers = matches
            price_list.append(numbers)
        
        ###3.만, 천의 단위가 한글로 쓰였을 경우 (ex. 5만 3천원)###  
        matches=re.findall(patterns[2],parsed_data[k]['events'][i]['message'])
        if matches:
            numbers = matches
            for number in range(len(numbers)):
                numbers[number]=numbers[number].split("만")
                numbers[number]=str(int(numbers[number][0])*10000+int("".join([k for k in numbers[number][-1] if k !='천']))*1000)
            price_list.append(numbers)
        
        ###4.만 의 단위가 한글로 쓰였을 경우 (ex.3만원, 12만원)###      
        matches=re.findall(patterns[3],parsed_data[k]['events'][i]['message'])
        if matches:
            numbers = matches
            for number in range(len(numbers)):
                numbers[number]="".join([k for k in numbers[number] if k !='만'])
                numbers[number]=str(int(numbers[number])*10000)
            price_list.append(numbers)
        
        ###뽑혀진 가격 정보 출력###        
        if len(price_list) == 1:
            price_list.append(None) #아무런 가격이 뽑히지 않았던 경우
        price_lists.append(price_list)
    
    ###구매자의 최종 제안 금액 출력###
    answer=[]
    for price in price_lists:
        if price[0]=='구매자':
            if price [1] != None:
                min_compare=[]
                for k in range(1,len(price)):
                    min_compare.append(min(price[k]))
                answer.append(min_compare)
    if answer != []:
        last_price=str(min(answer[-1]))
    else:
        last_price = '0'
    last_prices.append(" ##<"+last_price+">## ")
    last_price=""

#### 4.3 Data 변경

In [ ]:
###Data 변경###
for k in range(len(last_prices)):
    #마지막 발화자가 판매자인 경우
    if parsed_data[k]["events"][-1]["role"]=='판매자':
        parsed_data[k]["events"].append({'role':'구매자','message':last_prices[k]})
    #마지막 발화자가 구매자인 경우
    else:
        parsed_data[k]["events"][-1]['message']=last_prices[k]

#### 4.4 Noise Data 제거

In [7]:
###판매가가 0원이었거나 거래 중 0원을 암시한 경우 제거###
wrong_list = []
for k in range(len(parsed_data)):
    if parsed_data[k]["events"][-1]["message"] == " ##<0>## ":
        wrong_list.append(k)
    if parsed_data[k]["price"] == 0:
        wrong_list.append(k)
wrong_list = list(set(wrong_list))
wrong_list = sorted(wrong_list,reverse=True)
for wrong_index in wrong_list:
    del parsed_data[wrong_index]

#### 4.5 Data 저장

In [ ]:
new_file_path = f'/opt/final_merge.json' #YOUR PATH
with open(new_file_path, 'w', encoding='utf-8') as f:
    json.dump(parsed_data, f, ensure_ascii=False, indent=4)